What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [8]:
using Pkg
Pkg.activate("../../../")
# Pkg.instantiate()
using Globtim
# # Constants and Parameters
d = 1 # Initial Degree 
const n, a, b = 2, 11, 10 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters # Delta used to be too big 
const tol_l2 = 1e-4            # Define the tolerance for the L2-norm
const sample_scale = 1.0

f = Deuflhard # Objective function

  Activating project at `~/Globtim.jl`


Deuflhard (generic function with 1 method)

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [9]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, sample_scale) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        println("Degree :$d ")
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end
println("current L2-norm: ", poly_approx.nrm)
println("Number of samples: ", poly_approx.N)

current L2-norm: 0.7126746385412904
Number of samples: 20
current L2-norm: 0.2643263601556228
Number of samples: 36
current L2-norm: 0.16828389476537675
Number of samples: 56
current L2-norm: 0.05095097972974844
Number of samples: 79
current L2-norm: 0.03821589210464418
Number of samples: 105
current L2-norm: 0.011678441767439998
Number of samples: 134
current L2-norm: 0.009413714499985898
Number of samples: 166
current L2-norm: 0.0018310409855694131
Number of samples: 201
current L2-norm: 0.0015451932338800035
Number of samples: 238
current L2-norm: 0.0004709182424927247
Number of samples: 278
current L2-norm: 0.0004089177653992807
Number of samples: 320
current L2-norm: 0.00018404396152647953
Number of samples: 364
current L2-norm: 0.0001629465042998042
Number of samples: 411
attained the desired L2-norm: 3.3416987092232604e-5
Degree :14 
current L2-norm: 3.3416987092232604e-5
Number of samples: 460


We check the distribution of the magnitudes of the coefficients of the approximant

In [10]:
using Plots 
Plots.histogram(abs.(poly_approx.coeffs))

ArgumentError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.

We now expand the approximant computed in the tensorized Chebyshev basis into standard monomial basis and construct the system of partials for homotopy continuation. 

In [ ]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d)) # Convert coefficients to Float64 for homotopy continuation
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)

Solve the system of partial derivatives using HomotopyContinuation. The coefficients have to be Float64. 

In [ ]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

Sort through the critical points, make sure they fall into the domain of definition. Make them into a Dataframe.

In [6]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z= h_z); # Create a DataFrame

We proceed to generate the plot of the critical points over the sample set $\mathcal{S}$.

In [ ]:
# using PlotlyJS, Colors

# coords = poly_approx.scale_factor * poly_approx.grid # Extract coordinates and function values
# z_coords = poly_approx.z

# if size(coords)[2] == 2 # Plot the 3D scatter plot if the dimensions are 2
#     scatter_trace = PlotlyJS.scatter3d(
#         x=coords[:, 1],
#         y=coords[:, 2],
#         z=z_coords,
#         mode="markers",
#         marker=attr(
#             size=1,
#             color=z_coords,
#             colorscale="Viridis"
#         ),
#         name="Sampled Data"
#     )

#     # Create the scatter3d trace
#     # Had to switch the coordinates of the critical points to match the surface plot for some reason. 
#     crit_pts = PlotlyJS.scatter3d(
#         x=df.y,
#         y=df.x,
#         z=df.z,
#         mode="markers",
#         marker=attr(
#             size=10,
#             color="red"
#         ),
#         name="Exact approximant critical points"
#     )

#     layout = Layout(
#         title="Deuflhard Sample Points",
#         scene=attr(
#             xaxis=attr(title="X-axis"),
#             yaxis=attr(title="Y-axis"),
#             zaxis=attr(title="Z-axis")),
#         height=1200
#     )
#     plt1 = Plot([scatter_trace, crit_pts],layout)
# end


In [ ]:
using GLMakie
GLMakie.activate!()
GLMakie.closeall() # close any open screen

function peaks(; n=49)
    x = LinRange(-3, 3, n)
    y = LinRange(-3, 3, n)
    a = 3 * (1 .- x') .^ 2 .* exp.(-(x' .^ 2) .- (y .+ 1) .^ 2)
    b = 10 * (x' / 5 .- x' .^ 3 .- y .^ 5) .* exp.(-x' .^ 2 .- y .^ 2)
    c = 1 / 3 * exp.(-(x' .+ 1) .^ 2 .- y .^ 2)
    return (x, y, a .- b .- c)
end
x, y, z = peaks()
with_theme(theme_dark()) do
    fig = Figure(size=(1200, 800))
    ax1 = Axis(fig[1, 1], aspect=1)
    ax2 = Axis3(fig[1, 2]; aspect=(1, 1, 0.7), perspectiveness=0.5)
    axs = [ax1, ax2]
    cmap = :diverging_bkr_55_10_c35_n256
    contourf!(axs[1], x, y, z; levels=0.1:0.08:0.9, mode=:relative,
        colormap=cmap)
    # bug, colormap cannot be transparent
    contourf!(axs[2], x, y, z; levels=14, colormap=cmap)
    contour3d!(axs[2], x, y, z; levels=14, colormap=cmap,
        transparency=true, linewidth=5)
    limits!(axs[1], -3, 3, -3, 3)
    hidedecorations!.(axs; grid=false)
    fig
end

In [ ]:
println("Degree: $d")
println("current L2-norm: ", poly_approx.nrm)
println("Number of samples: ", poly_approx.N)
# savefig(plt1, "../../data/figures/Deuflhard/Exact_Deuflhard/3d_Deuflhard.html")